In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, skew
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score,cross_val_predict
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn import svm



In [2]:
train=pd.read_excel('D:\Data Science\Data Science\Data Trained\Assignments and Projects\Projects\Flight_Train.xlsx')



In [3]:
test=pd.read_excel('D:\Data Science\Data Science\Data Trained\Assignments and Projects\Projects\Flight_Test.xlsx')

In [4]:
train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [5]:
len(train)

10683

In [6]:
len(test)

2671

In [7]:
test.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [8]:
dataset=train.append(test)

C:\Users\SANDY1\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [9]:
dataset.head()

,Additional_Info,Airline,Arrival_Time,Date_of_Journey,Dep_Time,Destination,Duration,Price,Route,Source,Total_Stops
0,No info,IndiGo,01:10 22 Mar,24/03/2019,22:20,New Delhi,2h 50m,3897.0,BLR → DEL,Banglore,non-stop
1,No info,Air India,13:15,1/05/2019,05:50,Banglore,7h 25m,7662.0,CCU → IXR → BBI → BLR,Kolkata,2 stops
2,No info,Jet Airways,04:25 10 Jun,9/06/2019,09:25,Cochin,19h,13882.0,DEL → LKO → BOM → COK,Delhi,2 stops
3,No info,IndiGo,23:30,12/05/2019,18:05,Banglore,5h 25m,6218.0,CCU → NAG → BLR,Kolkata,1 stop
4,No info,IndiGo,21:35,01/03/2019,16:50,New Delhi,4h 45m,13302.0,BLR → NAG → DEL,Banglore,1 stop


In [10]:
dataset.shape

(13354, 11)

In [11]:
dataset.dtypes

Additional_Info     object
Airline             object
Arrival_Time        object
Date_of_Journey     object
Dep_Time            object
Destination         object
Duration            object
Price              float64
Route               object
Source              object
Total_Stops         object
dtype: object

In [12]:
dataset['Date']=dataset['Date_of_Journey'].str.split('/').str[0]
dataset['Month']=dataset['Date_of_Journey'].str.split('/').str[1]
dataset['Year']=dataset['Date_of_Journey'].str.split('/').str[2]

In [13]:
dataset['Date']=dataset['Date'].astype(int)
dataset['Month']=dataset['Month'].astype(int)
dataset['Year']=dataset['Year'].astype(int)

In [14]:
dataset=dataset.drop(['Date_of_Journey'], axis=1)

In [15]:
dataset['Arrival_Time']=dataset['Arrival_Time'].str.split(' ').str[0]


In [16]:
dataset.isna().sum()

Additional_Info       0
Airline               0
Arrival_Time          0
Dep_Time              0
Destination           0
Duration              0
Price              2671
Route                 1
Source                0
Total_Stops           1
Date                  0
Month                 0
Year                  0
dtype: int64

In [17]:
dataset['Total_Stops']=dataset['Total_Stops'].fillna('1 stop')

In [18]:
dataset['Total_Stops']=dataset['Total_Stops'].replace('non-stop','0 stop')

In [19]:
dataset['Total_Stops']=dataset['Total_Stops'].str.split(' ').str[0]

In [20]:
dataset['Total_Stops']=dataset['Total_Stops'].astype(int)

In [23]:
dataset['Arrival_Hour']=dataset['Arrival_Time'].str.split(':').str[0]
dataset['Arrival_Minute']=dataset['Arrival_Time'].str.split(':').str[1]


In [24]:
dataset['Arrival_Hour']=dataset['Arrival_Hour'].astype(int)
dataset['Arrival_Minute']=dataset['Arrival_Minute'].astype(int)

In [25]:
dataset=dataset.drop(['Arrival_Time'], axis=1)

In [26]:
dataset.columns

Index(['Additional_Info', 'Airline', 'Dep_Time', 'Destination', 'Duration',
       'Price', 'Route', 'Source', 'Total_Stops', 'Date', 'Month', 'Year',
       'Arrival_Hour', 'Arrival_Minute'],
      dtype='object')

In [27]:
dataset['Dep_Hour']=dataset['Dep_Time'].str.split(':').str[0]
dataset['Dep_Minute']=dataset['Dep_Time'].str.split(':').str[1]
dataset['Dep_Hour']=dataset['Dep_Hour'].astype(int)
dataset['Dep_Minute']=dataset['Dep_Minute'].astype(int)


In [28]:
dataset=dataset.drop(['Dep_Time'], axis=1)

In [29]:
dataset['Route1']=dataset['Route'].str.split('->').str[0]
dataset['Route2']=dataset['Route'].str.split('->').str[1]
dataset['Route3']=dataset['Route'].str.split('->').str[2]
dataset['Route4']=dataset['Route'].str.split('->').str[3]
dataset['Route5']=dataset['Route'].str.split('->').str[4]

In [30]:
dataset['Price'].fillna(dataset['Price'].mean(), inplace=True)

In [31]:
dataset['Route1'].fillna('None', inplace=True)
dataset['Route2'].fillna('None', inplace=True)
dataset['Route3'].fillna('None', inplace=True)
dataset['Route4'].fillna('None', inplace=True)
dataset['Route5'].fillna('None', inplace=True)

In [32]:
dataset.describe()

,Price,Total_Stops,Date,Month,Year,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute
count,13354.000000,13354.000000,13354.000000,13354.000000,13354.0,13354.000000,13354.000000,13354.000000,13354.000000
mean,9087.064121,0.826045,13.389846,4.710574,2019.0,13.396061,24.664146,12.513254,24.507264
std,4124.447805,0.674608,8.439060,1.165622,0.0,6.896145,16.559723,5.736273,18.832385
min,1759.000000,0.000000,1.000000,3.000000,2019.0,0.000000,0.000000,0.000000,0.000000
25%,6135.250000,0.000000,6.000000,3.000000,2019.0,8.000000,10.000000,8.000000,5.000000
50%,9087.064121,1.000000,12.000000,5.000000,2019.0,14.000000,25.000000,11.000000,25.000000
75%,11087.000000,1.000000,21.000000,6.000000,2019.0,19.000000,35.000000,18.000000,40.000000
max,79512.000000,4.000000,27.000000,6.000000,2019.0,23.000000,55.000000,23.000000,55.000000


In [33]:
dataset=dataset.drop(['Route', 'Duration'], axis=1)


In [34]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()


In [35]:
dataset["Additional_Info"]=le.fit_transform(dataset["Additional_Info"])
dataset["Airline"]=le.fit_transform(dataset["Airline"])
dataset["Destination"]=le.fit_transform(dataset["Destination"])
dataset["Source"]=le.fit_transform(dataset["Source"])
dataset['Route1']=le.fit_transform(dataset['Route1'])
dataset['Route2']=le.fit_transform(dataset['Route2'])
dataset['Route3']=le.fit_transform(dataset['Route3'])
dataset['Route4']=le.fit_transform(dataset['Route4'])
dataset['Route5']=le.fit_transform(dataset['Route5'])

In [36]:
dataset_train=dataset[0:10683]
dataset_test=dataset[10683:]
dataset_test=dataset_test.drop(['Price'], axis=1)

In [37]:
X=dataset_train.drop(['Price'], axis=1)
y=dataset_train.Price

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [39]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url = "http://i.imgur.com/QBuDOjs.jpg")

In [40]:
lr=LinearRegression()

In [41]:
kfolds=KFold(n_splits=50, shuffle=True, random_state=100)

In [42]:
def cv_rmse(model):
    
    rmse=np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kfolds))
    
    return(rmse)
                 

In [43]:

lr_model=make_pipeline(RobustScaler(), lr).fit(X=X_train, y=y_train)

cv_rmse(lr_model).mean()

3441.661564025337